# Class and Records
What is a record?
- Indroduced in C# 9
- reference type for encapsulating immutable types 
    - comes from functional programming where immutablilty is king
        - data is kept seperate from logic in immutable types called records
- C# 10 introduces record struct's

What is the motivation for having a record?
- Immutable by default
    - concise syntax for creating immutable properties
- Value semantics
    - variables are equal based on value, rather than reference
    - this is often what you want
        - but is non trivial to achieve with regular reference type

In [8]:
class Person
{
    public int Age {get; set;}
    public string Name {get; set;}

    public Person() { }

    public Person(int age, string name)
    {
        Age = age;
        Name = name;
    }
}

In [9]:
var q = new Person(1, "Quinten");
var q1 = new Person 
{
    Age = 1, 
    Name = "Quinten"
};
Console.WriteLine($"Person {{Age = {q.Age}, Name = {q.Name}}}");
Console.WriteLine($"Person {{Age = {q1.Age}, Name = {q1.Name}}}");

// but this is false even though we are trying to represent the same Person.
Console.WriteLine($"{q == q1}");

// and this is mutable :(
q.Name = "Bob";
Console.WriteLine($"Person {{Age = {q.Age}, Name = {q.Name}}}");

Person {Age = 1, Name = Quinten}
Person {Age = 1, Name = Quinten}
False
Person {Age = 1, Name = Bob}


Now its immutable

In [10]:
class Person
{
    public int Age {get; private set;}
    public string Name {get; private set;}

    public Person() { }

    public Person(int age, string name)
    {
        Age = age;
        Name = name;
    }
}

but equality is still not structural

In [11]:
var q = new Person(1, "Quinten");
var q1 = new Person(1, "Quinten");
Console.WriteLine($"Person {{Age = {q.Age}, Name = {q.Name}}}");
Console.WriteLine($"Person {{Age = {q1.Age}, Name = {q1.Name}}}");
Console.WriteLine($"{q == q1}");

Person {Age = 1, Name = Quinten}
Person {Age = 1, Name = Quinten}
False


We could implement IEquatable and all the operators... or

In [12]:
record Person(int Age, string Name);

var q = new Person(1, "Quinten");
var q1 = new Person(1, "Quinten");
Console.WriteLine(q);
Console.WriteLine(q1);
Console.WriteLine($"{q == q1}");

Person { Age = 1, Name = Quinten }
Person { Age = 1, Name = Quinten }
True


### But I like object initialization syntax
The `init` keyword allows you to create an immutable property that can be `initialized` during contruction.

In [13]:
record Person
{
    public string Name {get; init;}
    public int Age {get; init;}
}

var q = new Person
{
    Age = 1, 
    Name = "Quinten"
};
var q1 = new Person
{
    Age = 1, 
    Name = "Quinten"
};
Console.WriteLine(q);
Console.WriteLine(q1);
Console.WriteLine($"{q == q1}");

Person { Name = Quinten, Age = 1 }
Person { Name = Quinten, Age = 1 }
True


In [14]:
q.Name = "Bob";

Error: (1,1): error CS8852: Init-only property or indexer 'Person.Name' can only be assigned in an object initializer, or on 'this' or 'base' in an instance constructor or an 'init' accessor.

Using `required` we can still get constructor like constraints, with `init` syntax.

In [15]:
record Person
{
    public required string Name {get; init;}
    public required int Age {get; init;}
    public string Description {get;init;} = string.Empty;
}

var q = new Person
{
    Name = "Quinten",
    Age = 1
};

In [16]:
var bob = new Person {
    Age = 1,
    Description = "Just a nice guy"
};

Error: (1,15): error CS9035: Required member 'Person.Name' must be set in the object initializer or attribute constructor.

### What else does this enable?
Nondestructive mutation
- The result of a `with` expression is a shallow copy

In [17]:
var bob = q with {Name = "Bob", Description = "Just a nice guy"};
Console.WriteLine(q); // we still have q :)
Console.WriteLine(bob); // which means we have timetravel
Console.WriteLine(q == bob) // and at this point, they are no longer the same person

Person { Name = Quinten, Age = 1, Description =  }
Person { Name = Bob, Age = 1, Description = Just a nice guy }
False


^ did you notice the built in formatting? ^

### Serialization
It just works

In [18]:
using System.Text.Json.Serialization;

Console.WriteLine(JsonSerializer.Serialize(q));

Error: (3,19): error CS0103: The name 'JsonSerializer' does not exist in the current context

You can still decorate the consise syntax with attributes

In [19]:
using System.Text.Json.Serialization;

record Person(
    [property: JsonPropertyName("person_age")] int Age, 
    [property: JsonPropertyName("person_name")] string Name
);

var q = new Person(1,"Quinten");
Console.WriteLine(System.Text.Json.JsonSerializer.Serialize(q));

{"person_age":1,"person_name":"Quinten"}


### So what exactly is a record?
syntax sugar for a class that implements `IEquatable<T>`

In [20]:
class Person : IEquatable<Person>
{
    private readonly string name;
    private readonly int age;

    protected virtual Type EqualityContract
    {
        get
        {
            return typeof(Person);
        }
    }
    public string Name
    {
        get
        {
            return name;
        }
        init
        {
            name = value;
        }
    }
    public int Age
    {
        get
        {
            return age;
        }
        init
        {
            age = value;
        }
    }
    public Person(){}
    public Person(int age, string name)
    {
        this.age = age;
        this.name = name;
    }
    public override string ToString()
    {
        StringBuilder stringBuilder = new StringBuilder();
        stringBuilder.Append("Person");
        stringBuilder.Append(" { ");
        if (PrintMembers(stringBuilder))
        {
            stringBuilder.Append(" ");
        }
        stringBuilder.Append("}");
        return stringBuilder.ToString();
    }
    protected virtual bool PrintMembers(StringBuilder builder)
    {
        builder.Append("Name");
        builder.Append(" = ");
        builder.Append((object)Name);
        builder.Append(", ");
        builder.Append("Age");
        builder.Append(" = ");
        builder.Append((object)Age);
        return true;
    }
    public static bool operator !=(Person left, Person right)
    {
        return !(left == right);
    }
    public static bool operator ==(Person left, Person right)
    {
        return (object)left == right || ((object)left != null && left.Equals(right));
    }
    public override int GetHashCode()
    {
        return (EqualityComparer<Type>.Default.GetHashCode(EqualityContract) * -1521134295 + EqualityComparer<string>.Default.GetHashCode(name)) * -1521134295 + EqualityComparer<int>.Default.GetHashCode(Age);
    }
    public override bool Equals(object obj)
    {
        return Equals(obj as Person);
    }
    public virtual bool Equals(Person other)
    {
        return (object)this == other || ((object)other != null && EqualityContract == other.EqualityContract && EqualityComparer<string>.Default.Equals(name, other.name) && EqualityComparer<int>.Default.Equals(Age, other.Age));
    }
    public virtual Person Clone()
    {
        return new Person(this);
    }
    protected Person(Person original)
    {
        name = original.name;
        age = original.age;
    }
    public void Deconstruct(out string Name, out int Age)
    {
        Name = this.Name;
        Age = this.Age;
    }
}

var q = new Person(1, "Quinten");
var q1 = new Person {Age = 1, Name = "Quinten"};
Console.WriteLine(q);
Console.WriteLine(q1);
Console.WriteLine(q == q1);


Person { Name = Quinten, Age = 1 }
Person { Name = Quinten, Age = 1 }
True


In [21]:
q.Name = "Bob";

Error: (1,1): error CS8852: Init-only property or indexer 'Person.Name' can only be assigned in an object initializer, or on 'this' or 'base' in an instance constructor or an 'init' accessor.

### So, Class or Record?
record, the answer is record

when creating data objects, use a record

In [22]:
record Greeting(DateTime GeneratedAt, string Message);
record Person(string Name);

var q = new Person("Quinten");

when creating service objects to encapsulate functionality, use a class

In [23]:
class MyGreeterService 
{
    public Greeting GreetPerson(Person person) => 
        new Greeting(
            DateTime.UtcNow, 
            $"Hello, {person.Name}");
}

var greeting = new MyGreeterService().GreetPerson(q);
Console.WriteLine(greeting);

Greeting { GeneratedAt = 3/27/2023 3:11:11 PM, Message = Hello, Quinten }


# Introduction to Pattern Matching

Pattern matching in C# lets you test expressions for certain characteristics and take actions based on the first matching pattern. It can help you check for null values, types, and enum values more concisely and safely.

`is`

In [24]:
class Circle {
    public float Diameter => 1;
}

Circle circle;

Console.WriteLine(circle.Diameter);

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at Submission#25.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

pattern matching to save the day

In [25]:
Console.WriteLine(circle is Circle c ? c.Diameter : 0f); 
Console.WriteLine(circle is not null ? circle.Diameter : 0f); 

0
0


You should prefer pattern matching with `is` wherever possible, especially for `null` checks.
This is because pattern matching is more efficient than equality checks.
- there is no risk of overridden behavior
- overloads do not need to run
- it only checks exactly what is asked

`switch` (expression)

consice syntax for branching expression logic based on the shape of the data

In [26]:
string WaterState(int tempInFahrenheit) =>
    tempInFahrenheit switch
    {
        (> 32) and (< 212) => "liquid",
        < 32 => "solid",
        > 212 => "gas",
        32 => "solid/liquid transition",
        212 => "liquid / gas transition",
    };

Console.WriteLine(WaterState(1));
Console.WriteLine(WaterState(33));
Console.WriteLine(WaterState(220));

solid
liquid
gas


In [27]:
public record Order(int Items, decimal Cost);

public decimal CalculateDiscount(Order order) =>
    order switch
    {
        { Items: > 10, Cost: > 1000.00m } => 0.10m,
        { Items: > 5, Cost: > 500.00m } => 0.05m,
        { Cost: > 250.00m } => 0.02m,
        null => throw new ArgumentNullException(nameof(order), "Can't calculate discount on null order"),
        var someObject => 0m,
    };

var orders = new List<Order> { new Order(1, 100m), new Order(20,1000m) };
orders
    .Select(CalculateDiscount)
    .ToList()
    .ForEach(Console.WriteLine);

0
0.05


In [28]:
var input = """
04-01-2020,DEPOSIT,Initialdeposit,2250.00
04-15-2020,DEPOSIT,Refund,125.65
04-18-2020,DEPOSIT,Paycheck,825.65
04-22-2020,WITHDRAWAL,Debit,Groceries,255.73
05-01-2020,WITHDRAWAL,#1102,Rent,apt,2100.00
05-02-2020,INTEREST,0.65
05-07-2020,WITHDRAWAL,Debit,Movies,12.57
04-15-2020,FEE,5.55
""";

// bonus feature, _ the discard
decimal ParseTransaction(string[] transaction) => 
    transaction switch
    {
        [_, "DEPOSIT", _, var amount]     => decimal.Parse(amount),
        [_, "WITHDRAWAL", .., var amount] => -decimal.Parse(amount),
        [_, "INTEREST", var amount]       => decimal.Parse(amount),
        [_, "FEE", var fee]               => -decimal.Parse(fee),
        _                                 => throw new InvalidOperationException($"Record {string.Join(", ", transaction)} is not in the expected format!"),
    };

var records = 
    from row in input.Split('\n')
    select row.Split(',');

decimal balance = 0m;
foreach (string[] transaction in records)
{
    balance += ParseTransaction(transaction);
    Console.WriteLine($"Record: {string.Join(", ", transaction)}, New balance: {balance:C}");
}

Record: 04-01-2020, DEPOSIT, Initialdeposit, 2250.00, New balance: $2,250.00
Record: 04-15-2020, DEPOSIT, Refund, 125.65, New balance: $2,375.65
Record: 04-18-2020, DEPOSIT, Paycheck, 825.65, New balance: $3,201.30
Record: 04-22-2020, WITHDRAWAL, Debit, Groceries, 255.73, New balance: $2,945.57
Record: 05-01-2020, WITHDRAWAL, #1102, Rent, apt, 2100.00, New balance: $845.57
Record: 05-02-2020, INTEREST, 0.65, New balance: $846.22
Record: 05-07-2020, WITHDRAWAL, Debit, Movies, 12.57, New balance: $833.65
Record: 04-15-2020, FEE, 5.55, New balance: $828.10
